## Analise logs SATD

Funcionalidades: 
    
    
    Analise do gprof 
    
        Por aproximação
               Por vídeo e configuração
                -- Funções mais utilizadas (mais chamadas) (por QP)
                   - com gráfico
                -- Funções que gastam mais tempo (por QP)


    Analise do VVC log
        
        Por aproximação 
            -- bitrate
            -- PSNR
            -- tempo gasto 
    

In [1]:
from matplotlib import pyplot as plt
from IPython.display import clear_output,display, HTML
import numpy as np
import pandas as pd
import ipywidgets as widgets
import os
import glob

In [4]:
#CONSTANTS
GPROF_DIR = "../outputTests/gprof_log/"
VVCLOG_DIR =  "../outputTests/vvc_log/"
#pegar gprof e vvc juntos, pq o log do vvc é pequeno


#CLASS
class Video:
    def __init__(self, title):
        self.title = title 
        self.time = None
        self.m_used_functions = []
        self.BD_rate = None
        self.PSNR = [] 
        self.QP = None 
        self.config = None 
        
        def set_time(self,time):
            self.time = time
        
        def get_time():
            return(self.time)
            
        def add_function(self, function):
            self.m_used_functions.append(function)
        
        def get_functions(self):
            return(functions)
        
        def set_bdrate(self, bdrate):
            self.BD_rate = bdrate
            
        def get_bdrate(self):
            return(self.BD_rate)
            
        def set_PSNR(self, PSNR):
            self.PSNR = PSNR
            
        def get_PSNR(self):
            return(self.PSNR)
            
        def set_QP(self, QP):
            self.QP = QP
          
        def get_QP(self):
            return(self.QP)
        
        def set_config(self, config):
            self.config = config
            
        def get_config(self)
            return(self.config)    


class Aproximation
    def __init__(self, size, aprox_type):
        self.size = size
        self.aprox_type = aprox_type
        self.videos_qp22 = []
        self.videos_qp27 = []
        self.videos_qp32 = []
        self.videos_qp37 = []


#FUNCTIONS SYSTEM
def on_button_click(b):
    #selected_values = search_aproximations(dropdown.value, "-", GPROF_DIR)

    #for value in selected_values:
    #    value = GPROF_DIR + value
    #    print(value + '\n')

    #listando todos os videos dentro das pastas das aproximacoes do valor aproximado
    values_analised = analise(dropdown.value, GPROF_DIR, "")    
#list all matrices
def list_matrices(terminator, folder):
    items = os.listdir(folder)
    names = []
    for item in items:
        if os.path.isdir(os.path.join(folder, item)):
            prefix = item.split(terminator)[0]  
            if prefix not in names:  
                names.append(prefix)
    return names
    
#list all aproximations of a matrix researched
def search_aproximations(prefix, terminator, folder):
    items = os.listdir(folder)
    names = []
    for item in items:
        if os.path.isdir(os.path.join(folder, item)):
            prefix_item = item.split(terminator)[0]
            if prefix_item == prefix: 
                names.append(item)
    return names
    
#FUNCTIONS GPROF ANALISE
#returns the most used functions list 
def m_used_functions(file_gprof, number_of_itens):
    #aqui analisar o arquivo do gprof e retornar as mais usadas
    return void; 

#time to encode
def used_time(file_gprof):
    #aqui analisar o arquivo do gprof e retornar o tempo de execução 
    return void; 
    
#make the table according the param
def table(videos, param, qps, aproximations):
        #display(tf)
        return void


    
#returns the analise
def analise(matrix, folder, config):
    #find all aproximations for the matrix
    aproximations = search_aproximations(matrix, "-",folder)
    
    #find all videos for each aproximation
    for aproximation in aproximations:
        aproximation_path = folder + aproximation + "/"
        videos = os.listdir(aproximation_path);
        
        #imprime tabela dos videos
        df = pd.DataFrame(videos, columns=[aproximation])
        display(df)

    
    
    
    
####    
# Dropdown
options_drop = list_matrices("-", GPROF_DIR)
dropdown = widgets.Dropdown(options=options_drop)

# Botão
botao = widgets.Button(description='Imprimir')
botao.on_click(on_button_click)

SyntaxError: expected ':' (1356355804.py, line 51)

In [3]:
# widgets
display(HTML("<h2>Analise gprof</h2>"))
display(dropdown, botao)

Dropdown(options=('Precise', '8x16', '16x8'), value='Precise')

Button(description='Imprimir', style=ButtonStyle())

,8x16--2
0,BasketballPass
1,RaceHorses2
2,BasketballDrive
3,Campfire
4,BQMall
5,FourPeople
6,RitualDance
7,RaceHorses
8,FoodMarket4
9,Johnny


,8x16--4
0,BasketballPass
1,RaceHorses2
2,BasketballDrive
3,Campfire
4,BQMall
5,FourPeople
6,RitualDance
7,RaceHorses
8,FoodMarket4
9,Johnny


,8x16--1
0,BasketballPass
1,RaceHorses2
2,BasketballDrive
3,Campfire
4,BQMall
5,FourPeople
6,RitualDance
7,RaceHorses
8,FoodMarket4
9,Johnny


,8x16--3
0,BasketballPass
1,RaceHorses2
2,BasketballDrive
3,Campfire
4,BQMall
5,FourPeople
6,RitualDance
7,RaceHorses
8,FoodMarket4
9,Johnny
